![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Start coding here...
df = pd.read_csv("bank_marketing.csv")
df.head(2)

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no


In [3]:
# Check the data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

In [4]:
# Task: Change "." to "_"
df['job'] = df['job'].str.replace('.', '-')

# Task: Change "." to "_" and "unknown" to np.NaN
df['education'] = df['education'].str.replace('.', '-')
df.loc[df['education'] == 'unknown', 'education'] = np.nan

# Task: Whether the client's credit is in default	Convert to boolean data type: 1 if "yes", otherwise 0
df['credit_default'] = df['credit_default'].astype(bool)

# Task: Whether the client has an existing mortgage (housing loan)	Convert to boolean data type: 1 if "yes", otherwise 0
df['mortgage'] = df['mortgage'].astype(bool)

In [5]:
# map month column to their appropriate integer values
# first make sure the month values are in consistent string format
df['month'] = df['month'].str.lower()

# map month to their appropriate integer values
month_map = {
    'january': 1, 'february': 2, 'march': 3, 'april': 4,
    'may': 5, 'june': 6, 'july': 7, 'august': 8,
    'september': 9, 'october': 10, 'november': 11, 'december': 12
}

In [6]:
# Task: Convert the month column to numeric values
df['month'] = df['month'].map(month_map)

In [7]:
# Task: Create a new column 'year' with constant value 2022
df['year'] = 2022

In [8]:
# Task: Convert to boolean data type: 1 if "success", otherwise 0.
df['previous_outcome'] = df['previous_outcome'].apply(lambda x: (x == "yes")).astype(bool)

# Task: Convert to boolean data type: 1 if "yes", otherwise 0.
df['campaign_outcome'] = df['campaign_outcome'].apply(lambda x: (x == "yes")).astype(bool)

# Task: Create from a combination of day, month, and a newly created year column (which should have a value of 2022); Format = "YYYY-MM-DD"
df['last_contact_date'] = pd.to_datetime(df[['year', 'month', 'day']], format='%Y-%m-%d')

In [9]:
# Create Client DataFrame
df_client = df[[
    'client_id',
    'age',
    'job',
    'marital',
    'education',
    'credit_default',
    'mortgage'
]]

In [10]:
# Create Campaign DataFrame
df_campaign = df[[
    'client_id',
    'number_contacts',
    'contact_duration',
    'previous_campaign_contacts',
    'previous_outcome',
    'campaign_outcome',
    'last_contact_date'
]]

In [11]:
# Create Economics DataFrame
df_economics = df[[
    'client_id',
    'cons_price_idx',
    'euribor_three_months'
]]

In [12]:
# Save DataFrame to client.csv file.
df_client.to_csv('client.csv', index=False)
# Save DataFrame to campaign.csv file.
df_campaign.to_csv('campaign.csv', index=False)
# Save DataFrame to economics.csv file.
df_economics.to_csv('economics.csv', index=False)